In [32]:
import datetime

filename = 'moonphasedf.csv'

def build_phases(filename):
    phases = {}
    f = open(filename)
    lines = f.readlines()
    f.close() 
    
    for i in range(1,len(lines)):
        try:
            date = datetime.datetime.strptime(lines[i].split(',')[1], '%Y %b %d')
            value = lines[i].split(',')[2]
        except Exception as e:
            print(e)
            continue

        if value not in phases:
            phases[value] = []
        phases[value].append(date)
        
    f.close()  
    
    return phases

def get_phase_from_day(day_string):
    try:
        date = datetime.datetime.strptime(day_string, '%Y %b %d')
    except Exception as e:
        print(e)
            
    for phase in phases:
        if date in phases[phase]:
            return phase
        else:
            return day_string + ' not found'
    
    
someday = '2012 Jan 01'
phases = build_phases(filename)

for phase in phases:
    print('{}: {}'.format(phase, len(phases[phase])))
    
get_phase_from_day(someday)

First Quarter: 546
Full Moon: 546
Last Quarter: 546
New Moon: 542


'First Quarter'